In [ ]:
# Importing necessary modules

from torch.distributions import MultivariateNormal
import matplotlib.pyplot as plt
from collections import deque
import torch.optim as optim
import gymnasium as gym
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import imageio
import torch

In [ ]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, action_std):
        """
        Initializes the Actor network.
        
        Args:
        - state_dim  : Dimension of the input state.
        - action_dim : Dimension of the output action.
        - action_std : Standard deviation for the action distribution (controls exploration).
        """
        super(Actor, self).__init__()

        # Define a simple feedforward neural network
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim), 
        )
        self.action_std = action_std

    def forward(self, state):
        """
        Forward pass of the Actor network.
        
        Args:
        - state: Input state tensor (batch_size x state_dim).
        
        Returns:
        - action_mean : Mean of the actions (batch_size x action_dim).
        - cov_matrix  : Covariance matrix   (batch_size x action_dim x action_dim).
        """
        action_mean = self.fc(state)  # Compute the action means from the network

        # Create a diagonal covariance matrix using the standard deviation squared
        cov_matrix = torch.diag(torch.full((action_mean.size(-1),), self.action_std**2)).unsqueeze(0)
        return action_mean, cov_matrix  # Return the mean and covariance for the action distribution


class Critic(nn.Module):
    def __init__(self, state_dim):
        """
        Initializes the Critic network.
        
        Args:
        - state_dim: Dimension of the input state.
        """
        super(Critic, self).__init__()

        # Define a feedforward neural network for state value estimation
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, state):
        """
        Forward pass of the Critic network.
        
        Args:
        - state: Input state tensor (batch_size x state_dim).
        
        Returns:
        - Value estimate: Scalar value representing the state value (batch_size x 1).
        """
        return self.fc(state)  # Compute and return the state value

In [ ]:
class PPO:
    def __init__(self, state_dim, action_dim, action_std, lr, gamma, eps_clip):
        """
        Initializes the PPO agent.
        
        Args:
        - state_dim  : Dimension of the input state.
        - action_dim : Dimension of the output action.
        - action_std : Standard deviation of the action distribution.
        - lr         : Learning rate for the optimizer.
        - gamma      : Discount factor for rewards.
        - eps_clip   : Clipping parameter for PPO (used in the policy update).
        """
        # Instantiate the Actor and Critic networks
        self.actor  = Actor(state_dim, action_dim, action_std)  # Policy network
        self.critic = Critic(state_dim)                         # Value network
        
        # Define optimizers for both networks - Using Adam for both
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=lr)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr)
        
        # Store other PPO-specific parameters
        self.gamma      = gamma       # Discount factor for calculating the return
        self.eps_clip   = eps_clip    # Clipping parameter for PPO to prevent large updates
        self.action_std = action_std  # Standard deviation for action exploration

    def select_action(self, state):
        """
        Selects an action based on the current state using the policy network.
        
        Args:
        - state: The current state (numpy array or tensor).
        
        Returns:
        - action: The action sampled from the policy (numpy array).
        - action_logprob: The log probability of the action (used in training).
        """
        with torch.no_grad():
            # Convert the state to a PyTorch tensor and add a batch dimension
            state = torch.FloatTensor(state).unsqueeze(0)
            
            # Pass the state through the Actor to get the action mean and covariance matrix
            action_mean, cov_matrix = self.actor(state)
            
            # Create a multivariate normal distribution for the action and sample the action from it
            dist           = MultivariateNormal(action_mean, cov_matrix)
            action         = dist.sample()
            action_logprob = dist.log_prob(action)
            
            # Return the action as a numpy array (remove the batch dimension) and its log probability
            return action.squeeze(0).numpy(), action_logprob

    def evaluate(self, states, actions):
        """
        Evaluates the actions and states to compute log probabilities, state values, and entropy.
        
        Args:
        - states  : A batch of input states  (tensor of shape batch_size x state_dim).
        - actions : A batch of actions taken (tensor of shape batch_size x action_dim).
        
        Returns:
        - action_logprobs : Log probabilities of the actions   (tensor of shape batch_size).
        - state_values    : Estimated state values             (tensor of shape batch_size x 1).
        - dist_entropy    : Entropy of the action distribution (tensor of shape batch_size).
        """
        # Pass the states through the Actor to get action means and covariance matrices
        action_mean, cov_matrix = self.actor(states)
        
        # Create a multivariate normal distribution using the action mean and covariance and 
        # Compute the log probabilities of the given actions
        dist            = MultivariateNormal(action_mean, cov_matrix)
        action_logprobs = dist.log_prob(actions)
        
        # Compute the entropy of the distribution (used to encourage exploration)
        dist_entropy = dist.entropy()
        
        # Pass the states through the Critic to get the state values
        state_values = self.critic(states)
        
        # Return the log probabilities, state values, and entropy
        return action_logprobs, state_values, dist_entropy


In [ ]:
def collect_rollout(env, agent, timesteps):
    """
    Collects a batch of environment interactions (rollout) using the current policy.
    
    Args:
    - env       : The gymnasium environment to interact with
    - agent     : The PPO agent containing the policy
    - timesteps : Number of timesteps to collect per rollout
    
    Returns:
    - states   : List of observed states
    - actions  : List of actions taken
    - rewards  : List of rewards received
    - logprobs : List of log probabilities of chosen actions
    """
    state = env.reset()[0]  # Reset environment and get initial state
    states, actions, rewards, logprobs = [], [], [], []
    
    for _ in range(timesteps):
        # Select action using current policy
        action, logprob = agent.select_action(state)
        
        # Take action in environment
        next_state, reward, done, _, _ = env.step(action)
        
        # Store transition data
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        logprobs.append(logprob)
        
        # Reset environment if episode is done, otherwise continue
        if done:
            state = env.reset()[0]
        else:
            state = next_state
            
    return states, actions, rewards, logprobs

In [ ]:
def compute_returns(rewards, gamma):
    """
    Computes discounted returns (cumulative future rewards) for each timestep.
    
    Args:
    - rewards : List of immediate rewards from the environment
    - gamma   : Discount factor for future rewards
    
    Returns:
    - returns : List of discounted returns for each timestep
    """
    returns = []
    discounted_sum = 0
    # Iterate through rewards in reverse order to compute returns
    for r in reversed(rewards):
        discounted_sum = r + gamma * discounted_sum
        returns.insert(0, discounted_sum)
    return returns

In [ ]:
def update(agent, states, actions, rewards, logprobs, gamma, eps_clip):
    """
    Updates the policy and value networks using the PPO algorithm.
    
    Args:
    - agent      : The PPO agent containing the networks
    - states     : Batch of observed states
    - actions    : Batch of actions taken
    - rewards    : List of discounted returns
    - logprobs   : Log probabilities of the actions taken
    - gamma      : Discount factor for rewards
    - eps_clip   : PPO clipping parameter
    """
    # Convert collected data to PyTorch tensors
    states       = torch.FloatTensor(states)
    actions      = torch.FloatTensor(actions)
    rewards      = torch.FloatTensor(compute_returns(rewards, gamma))
    old_logprobs = torch.FloatTensor(logprobs)

    # Perform multiple epochs of PPO updates
    for _ in range(10):  
        # Evaluate current policy and value estimates
        logprobs, state_values, dist_entropy = agent.evaluate(states, actions)
        
        # Calculate advantages (how much better/worse actions were than expected)
        advantages  = rewards - state_values.squeeze().detach()
        
        # Normalize advantages for stable training
        advantages  = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # Calculate PPO policy loss with clipping
        ratios      = torch.exp(logprobs - old_logprobs)  # Importance sampling ratios
        surr1       = ratios * advantages
        surr2       = torch.clamp(ratios, 1 - eps_clip, 1 + eps_clip) * advantages
        policy_loss = -torch.min(surr1, surr2).mean()

        # Calculate value loss and combine losses
        value_loss  = nn.MSELoss()(state_values.squeeze(), rewards)
        loss        = policy_loss + 0.5 * value_loss - 0.01 * dist_entropy.mean()

        # Perform gradient update
        agent.actor_optimizer.zero_grad()
        agent.critic_optimizer.zero_grad()
        loss.backward()
        agent.actor_optimizer.step()
        agent.critic_optimizer.step()

In [ ]:
def run_single_trial(trial_num, env_name):
    """
    Runs a single training trial of the PPO algorithm.
    
    Args:
    - trial_num : Current trial number (for tracking purposes)
    - env_name  : Name of the gymnasium environment to use
    
    Returns:
    - trial_rewards : List of rewards obtained during training
    - ppo           : Trained PPO agent
    """
    print(f"\nStarting Trial {trial_num + 1}/5")
    
    # Initialize environment and get dimensions
    env        = gym.make(env_name, render_mode="rgb_array")
    state_dim  = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    action_std = 0.5  # Standard deviation for exploration
    
    # Set random seeds for reproducibility
    torch.manual_seed(trial_num)
    np.random.seed(trial_num)
    
    # Initialize PPO agent and training parameters
    ppo                   = PPO(state_dim, action_dim, action_std, lr=3e-4, gamma=0.99, eps_clip=0.2)
    episodes              = 1000
    timesteps_per_rollout = 2048
    
    # Initialize tracking variables
    trial_rewards = []
    avg_rewards = deque(maxlen=100)
    
    # Main training loop
    for episode in tqdm(range(episodes), desc=f"Trial {trial_num + 1}"):
        # Collect experience and update policy
        states, actions, rewards, logprobs = collect_rollout(env, ppo, timesteps_per_rollout)
        update(ppo, states, actions, rewards, logprobs, gamma=0.99, eps_clip=0.2)
        
        # Track rewards
        episode_reward = sum(rewards)
        trial_rewards.append(episode_reward)
        avg_rewards.append(episode_reward)
        
        # Print progress every 10 episodes
        if episode % 10 == 0:
            avg_reward = np.mean(list(avg_rewards))
            print(f"\nTrial {trial_num + 1}, Episode {episode}, Average Reward: {avg_reward:.2f}")
    
    # Record demonstration video of trained agent
    try:
        frames    = []
        state     = env.reset()[0]
        done      = False
        steps     = 0
        max_steps = 500
        
        while not done and steps < max_steps:
            action, _ = ppo.select_action(state)
            state, _, done, _, _ = env.step(action)
            if steps % 2 == 0:  # Save every other frame to reduce video size
                frames.append(env.render())
            steps += 1
        
        video_filename = f"ppo_halfcheetah_trial_{trial_num + 1}.mp4"
        imageio.mimsave(video_filename, frames, fps=30)
        print(f"Video saved as '{video_filename}'")
    except Exception as e:
        print(f"Warning: Could not save video for trial {trial_num + 1}: {e}")
        
    env.close()
    return trial_rewards, ppo

In [ ]:
def plot_rewards_with_trials(all_trial_rewards, window=100):
    """
    Plots the training progress across all trials with confidence intervals.
    
    Args:
    - all_trial_rewards : List of rewards from each trial
    - window           : Window size for reward smoothing
    """
    plt.figure(figsize=(10, 6))
    
    # Convert to numpy array for statistical calculations
    rewards_array = np.array(all_trial_rewards)
    
    # Calculate statistics across trials
    mean_rewards = np.mean(rewards_array, axis=0)
    std_rewards = np.std(rewards_array, axis=0)
    
    # Calculate smoothed mean rewards using moving average
    smoothed_means = []
    for i in range(len(mean_rewards)):
        if i < window:
            smoothed_means.append(np.mean(mean_rewards[:i+1]))
        else:
            smoothed_means.append(np.mean(mean_rewards[i-window+1:i+1]))
    smoothed_means = np.array(smoothed_means)
    
    # Plot individual trials with low opacity
    for trial_rewards in all_trial_rewards:
        plt.plot(trial_rewards, alpha=0.1, color='gray')
    
    # Plot mean and confidence interval
    episodes = range(len(mean_rewards))
    plt.plot(episodes, smoothed_means, linewidth=2, color='blue', label='Mean Reward')
    plt.fill_between(episodes, 
                    smoothed_means - std_rewards, 
                    smoothed_means + std_rewards, 
                    color='blue', alpha=0.2, label='Standard Deviation')
    
    # Add plot details
    plt.xlabel('Episode')
    plt.ylabel('Average Return')
    plt.title('PPO Training Progress - HalfCheetah-v4')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('ppo_training_progress.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def main():
    """
    Main function to run multiple trials of PPO training and visualize results.
    """
    num_trials = 5
    all_trial_rewards = []
    env = "HalfCheetah-v4"
    
    # Run multiple trials
    for trial in range(num_trials):
        trial_rewards, _ = run_single_trial(trial, env)
        all_trial_rewards.append(trial_rewards)
        np.save(f'trial_rewards_{trial}.npy', trial_rewards)
    
    # Generate and save final visualization
    plot_rewards_with_trials(all_trial_rewards)
    print("\nTraining progress plot saved as 'ppo_training_progress.png'")

if __name__ == "__main__":
    main()


Starting Trial 1/5


Trial 1:   0%|          | 1/1000 [00:01<19:46,  1.19s/it]


Trial 1, Episode 0, Average Reward: -268.52


Trial 1:   1%|          | 11/1000 [00:10<15:07,  1.09it/s]


Trial 1, Episode 10, Average Reward: -356.38


...


Trial 5, Episode 980, Average Reward: 2075.00


Trial 5:  99%|█████████▉| 991/1000 [14:32<00:09,  1.00s/it]


Trial 5, Episode 990, Average Reward: 2211.04


Trial 5: 100%|██████████| 1000/1000 [14:42<00:00,  1.13it/s]


Video saved as 'ppo_halfcheetah_trial_5.mp4'

Training progress plot saved as 'ppo_training_progress.png'
